In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('Titanic-Dataset.csv')
df_clean = df.copy()

In [2]:
#Standardize column  names (Production Rule)

df_clean.columns =(
    df_clean.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df_clean.columns

Index(['passengerid', 'survived', 'pclass', 'name', 'sex', 'age', 'sibsp',
       'parch', 'ticket', 'fare', 'cabin', 'embarked'],
      dtype='object')

In [6]:
# Data type fixing (critical steps)

# convert categorical variables

cat_cols = ['sex', 'embarked']
df_clean[cat_cols] = df_clean[cat_cols].astype('category')

In [7]:
# Covert target variable

df_clean['survuived'] = df_clean['survived'].astype('int')

In [8]:
# missing value Treatment (MOST IMPORTANT)

# Age median (Robust to outliers)

df_clean['age'] = df_clean['age'].fillna(df_clean['age'].median())



In [9]:
# Embarked - mode (Most frequent)

df_clean['embarked'] = df_clean['embarked'].fillna(
    df_clean['embarked'].mode([0])
)

In [10]:
# Cabin - Drop (Too many missing, low buisness value)

df_clean.drop(columns=['cabin'], inplace=True)

In [11]:
# Duplicate Handling
df_clean.drop_duplicates(inplace=True)

In [13]:
# Outlier Treatment (Google style: Cap, not remove)

#Fare capping (IQR Method)

Q1 = df_clean['fare'].quantile(0.25)
Q3 = df_clean['fare'].quantile(0.75)
IQR = Q3 - Q3

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df_clean['fare'] = np.where(
    df_clean['fare'] > upper, upper,
    np.where(df_clean['fare'] < lower, lower,df_clean['fare'])
)


In [14]:
# Feature engineeering (cleaning + intelligence)

# family size

df_clean['family_size'] = (
    df_clean['sibsp'] + df_clean['parch'] + 1
)

In [16]:
# Is alone  flag

df_clean['is_alone'] = np.where(
    df_clean['family_size'] == 1, 1,0
)

In [17]:
#Encoding (For ML Readiness)

# Label Encoding

df_clean['sex'] = df_clean['sex'].map({'male':0, 'female':1})
df_clean['embarked']= df_clean['embarked'].map({'s':0, 'c':1, 'Q':2})

In [18]:
# Final Data Quality Check (Sanity)

df_clean.info()

df_clean.isnull().sum()

df_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   passengerid  891 non-null    int64   
 1   survived     891 non-null    int64   
 2   pclass       891 non-null    int64   
 3   name         891 non-null    object  
 4   sex          891 non-null    category
 5   age          891 non-null    float64 
 6   sibsp        891 non-null    int64   
 7   parch        891 non-null    int64   
 8   ticket       891 non-null    object  
 9   fare         891 non-null    float64 
 10  embarked     77 non-null     float64 
 11  survuived    891 non-null    int64   
 12  family_size  891 non-null    int64   
 13  is_alone     891 non-null    int64   
dtypes: category(1), float64(3), int64(8), object(2)
memory usage: 91.6+ KB


,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,embarked,survuived,family_size,is_alone
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.9104,NaN,0,2,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,31.0000,NaN,1,2,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,31.0000,NaN,1,2,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,1,1


In [19]:
# Save cleaned dataset (Production Step)

df_clean.to_csv('Titanic_cleaned.csv', index=False)